In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import string
import warnings
warnings.filterwarnings("ignore")

In [ ]:
train_data = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
#train_data.head()
test_data = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")
train_data.head()


Explore Data:

In [ ]:
train_data[train_data['target']==1].values[1]
x = train_data.target.value_counts()
print(x)
sns.barplot(x.index,x)
plt.gca().set_ylabel('Y-Axiz')
plt.gca().set_xlabel('X-Axiz')

In [ ]:
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(10,5))
tweet_len=train_data[train_data['target']==1]['text'].str.len()
ax1.hist(tweet_len,color='blue')
ax1.set_title('disaster tweets')
tweet_len=train_data[train_data['target']==0]['text'].str.len()
ax2.hist(tweet_len,color='green')
ax2.set_title('Not disaster tweets')
fig.suptitle('Characters in tweets')
plt.show()

Data Cleaning:

In [ ]:
#Text Preporcessing
#Convert LowerCase:
train_data["text_clean"] = train_data["text"].apply(lambda x: x.lower())
test_data["text_clean"] = test_data["text"].apply(lambda x: x.lower())

#Special Characters Removal:
train_data['text_clean']= train_data['text_clean'].str.replace('rt ',"").str.replace('@','').str.replace('#','').str.replace('[^\w\s]','').str.replace('[1-9]','')
test_data['text_clean']= test_data['text_clean'].str.replace('rt ',"").str.replace('@','').str.replace('#','').str.replace('[^\w\s]','').str.replace('[1-9]','')
# Remove URL
def remove_URL(text):
    return re.sub(r"https?://\S+|www\.\S+", "", text)

train_data["text_clean"] = train_data["text_clean"].apply(lambda x: remove_URL(x))
test_data["text_clean"] = test_data["text_clean"].apply(lambda x: remove_URL(x))

def remove_html(text):
    html = re.compile(r"<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});")
    return re.sub(html, "", text)

train_data["text_clean"] = train_data["text_clean"].apply(lambda x: remove_html(x))
test_data["text_clean"] = test_data["text_clean"].apply(lambda x: remove_html(x))

def remove_emojis(text):
    emoji_pattern = re.compile(
        '['
        u'\U0001F600-\U0001F64F'  # emoticons
        u'\U0001F300-\U0001F5FF'  # symbols & pictographs
        u'\U0001F680-\U0001F6FF'  # transport & map symbols
        u'\U0001F1E0-\U0001F1FF'  # flags (iOS)
        u'\U00002702-\U000027B0'
        u'\U000024C2-\U0001F251'
        ']+',
        flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

train_data["text_clean"] = train_data["text_clean"].apply(lambda x: remove_emojis(x))
test_data["text_clean"] = test_data["text_clean"].apply(lambda x: remove_emojis(x))

def remove_punct(text):
    return text.translate(str.maketrans('', '', string.punctuation))


train_data["text_clean"] = train_data["text_clean"].apply(lambda x: remove_punct(x))
test_data["text_clean"] = test_data["text_clean"].apply(lambda x: remove_punct(x))

def remove_non_ascii(text):
    return re.sub(r'[^\x00-\x7f]',r'', text)

train_data["text_clean"] = train_data["text_clean"].apply(lambda x: remove_non_ascii(x))
test_data["text_clean"] = test_data["text_clean"].apply(lambda x: remove_non_ascii(x))
print(train_data["text_clean"])
print(test_data["text_clean"])




In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

train_data['text'] = train_data['text'].apply(lambda x: ' '.join(term for term in x.split() if term not in stop_words))
test_data['text'] = train_data['text'].apply(lambda x: ' '.join(term for term in x.split() if term not in stop_words))






In [ ]:
#Filling missing values
train_data.isnull().sum()
test_data.isnull().sum()
train_data['location'] = train_data['location'].fillna('None')
train_data['keyword'] = train_data['keyword'].fillna('None')
test_data['location'] = test_data['location'].fillna('None')
test_data['keyword'] = test_data['keyword'].fillna('None')

In [ ]:
train_data.head()


# Dropping Un-necessary features:
#train_data = train_data.drop('id', axis=1)
train_data.drop(columns=['id'])
test_data.drop(columns=['id'])
#test_data = test_data.drop('id', axis=1)



In [ ]:
# Tokenization
import nltk
from nltk import TweetTokenizer

tokenizer = TweetTokenizer()

train_data['tokens'] = [tokenizer.tokenize(item) for item in train_data.text]
test_data['tokens'] = [ tokenizer.tokenize(item) for item in test_data.text]
print(train_data['tokens'])
print(test_data['tokens'])



In [ ]:
# Lemmatization

from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

def lemmatize_item(item):
    new_item = []
    for x in item:
        x = lemmatizer.lemmatize(x)
        new_item.append(x)
    return " ".join(new_item)

train_data['tokens'] = [lemmatize_item(item) for item in train_data.tokens]
test_data['tokens'] = [lemmatize_item(item) for item in test_data.tokens]
print(train_data['tokens'])

In [ ]:
# Vectorization

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()


X = vectorizer.fit_transform(train_data.text).toarray()
y = train_data['target']

In [ ]:
#Data Splitting
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)


In [ ]:
#Applying Model
# LOGISTIC REGRESSION
"""
from sklearn.linear_model import LogisticRegression

# Logistic Regression Model 
log = LogisticRegression(random_state = 0)

# Fitting data
log.fit(X_train, y_train)

# Predicting the data
log_pred = log.predict(X_test)

"""

In [ ]:
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
log = RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
log.fit(X_train,y_train)

log_pred=log.predict(X_test)

In [ ]:
"""
from sklearn.ensemble import AdaBoostClassifier

log = AdaBoostClassifier()
log.fit(X_train,y_train)

log_pred=log.predict(X_test)
"""

In [ ]:
# MODEL EVALUATION FOR LOGISTIC REGRESSION

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score,KFold

# Accuracy
acc = accuracy_score(y_test,log_pred)
print("\nACCURACY : ",acc)



In [ ]:
# KFold Cross Validation
kfold = KFold(n_splits=15)
results = cross_val_score(log, X, y,cv=kfold)
print("\nK-FOLD CROSS VALIDATION : ",results.mean())



In [ ]:
# Classification Report
clf_report = classification_report(y_test,log_pred)
print("\nCLASSIFICATION REPORT:\n", clf_report)



In [ ]:
# Confusion Matrix
print("\nCONFUSION MATRIX:")
sns.heatmap(confusion_matrix(y_test,log_pred),annot=True,fmt='g')
plt.show()


In [ ]:
test = vectorizer.transform(test_data.text).toarray()
pred = log.predict(test)
submission_df = pd.read_csv('../input/nlp-getting-started/sample_submission.csv')
submission_df['target'] = pred
submission_df.to_csv('submission.csv', index=False)
result = pd.read_csv('submission.csv')
print (result)
#result